In [ ]:
%pip install --upgrade --quiet langchain langchain-community langchain-openai langchain-experimental neo4j wikipedia tiktoken yfiles_jupyter_graphs

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 981.0 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 997.8/997.8 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 25.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.8/49.8 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.3/204.3 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.6/293.6 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 32.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.5/15.5 MB 42.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.4/139.4 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 389.8/389.8 kB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.4/140.4 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 361.5/361.5 kB 14.5 MB/s eta 0:00:00
  

In [ ]:
from openai import AzureOpenAI
from langchain_openai import AzureChatOpenAI
from langchain_experimental.graph_transformers import LLMGraphTransformer
from yfiles_jupyter_graphs import GraphWidget
from neo4j import GraphDatabase

API_KEY = ""
RESOURCE_ENDPOINT = ""
DEPLOYMENT_NAME = ""
ENDPOINT = f"{RESOURCE_ENDPOINT}/openai/deployments//chat/completions?api-version="
API_VERSION = ''

client = AzureChatOpenAI(
    api_key=API_KEY,
    api_version = API_VERSION,
    azure_endpoint = ENDPOINT,
    deployment_name = DEPLOYMENT_NAME
)

# LLMGraphTransformer: 텍스트 데이터를 그래프 형식으로 변환
llm_transformer = LLMGraphTransformer(
    llm=client
)

In [ ]:
from langchain.text_splitter import TokenTextSplitter
from langchain.document_loaders import WikipediaLoader
loader = WikipediaLoader(query='Elizabeth I').load()

# 'Elizabeth I' 주제의 위키피디아 문서 로드 -> 첫 세 개를 분할하여 그래프 문서로 변환.
# LLMGraphTransformer를 사용하여 분할된 문서를 그래프 데이터 구조로 변환
text_splitter = TokenTextSplitter(chunk_size=512, chunk_overlap=24)
documents = text_splitter.split_documents(loader[:3])
graph_documents = llm_transformer.convert_to_graph_documents(documents)

/usr/local/lib/python3.10/dist-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /usr/local/lib/python3.10/dist-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


In [ ]:
graph_documents

[GraphDocument(nodes=[Node(id='Elizabeth I', type='Person'), Node(id='England', type='Place'), Node(id='Ireland', type='Place'), Node(id='House Of Tudor', type='Organization'), Node(id='Henry Viii', type='Person'), Node(id='Anne Boleyn', type='Person'), Node(id='Third Succession Act 1543', type='Event'), Node(id='Edward Vi', type='Person'), Node(id='Lady Jane Grey', type='Person'), Node(id='Mary', type='Person'), Node(id='William Cecil', type='Person'), Node(id='Baron Burghley', type='Title'), Node(id='James Vi Of Scotland', type='Person'), Node(id='Mary, Queen Of Scots', type='Person'), Node(id='Church Of England', type='Organization'), Node(id='Elizabethan Religious Settlement', type='Event'), Node(id='Sir Francis Walsingham', type='Person'), Node(id='Video Et Taceo', type='Motto')], relationships=[Relationship(source=Node(id='Elizabeth I', type='Person'), target=Node(id='England', type='Place'), type='QUEEN_OF'), Relationship(source=Node(id='Elizabeth I', type='Person'), target=Node

In [ ]:
from langchain_community.graphs import Neo4jGraph

NEO4J_URI=''
NEO4J_USERNAME=''
NEO4J_PASSWORD=''
AURA_INSTANCEID=''
AURA_INSTANCENAME=''

# Neo4j 데이터베이스 연결
graph = Neo4jGraph(url=NEO4J_URI, username=NEO4J_USERNAME, password = NEO4J_PASSWORD)

In [ ]:
# 그래프 문서를 Neo4j 그래프에 추가
# baseEntityLabel=True: 기본 엔티티 라벨 사용
# include_source=True: 원본 데이터 포함
graph.add_graph_documents(
    graph_documents,
    baseEntityLabel=True,
    include_source=True
)

In [ ]:
# Cypher 쿼리 정의: Neo4j MENTIONS 관계를 가진 노드들을 매칭하여 반환
# s: 출발 노드, r: 관계 노드, t: 목적지 노드
default_cyper = "MATCH (s)-[r:!MENTIONS]->(t) RETURN s,r,t LIMIT 50"

In [ ]:
from yfiles_jupyter_graphs import GraphWidget #yFiles 그래프 위젯
from neo4j import GraphDatabase #Neo4j 데이터베이스 드라이버

# 구글 코랩 환경에서 커스텀 위젯 매니저 활성화.
try:
  import google.colab
  from google.colab import output
  output.enable_custom_widget_manager()
except:
  pass

In [ ]:
# Neo4j 그래프 데이터 시각화
def showGraph(cypher: str = default_cyper):
    driver = GraphDatabase.driver(
        uri = NEO4J_URI,
        auth = (NEO4J_USERNAME, NEO4J_PASSWORD)
    )
    session = driver.session()
    widget = GraphWidget(graph = session.run(cypher).graph())
    widget.node_label_mapping = 'id'
    display(widget)
    return widget

In [ ]:
showGraph()

GraphWidget(layout=Layout(height='800px', width='100%'))

GraphWidget(layout=Layout(height='800px', width='100%'))

In [ ]:
def store_query_in_graph(user_query: str, graph_data: list):
    # Neo4j 데이터베이스에 연결
    driver = GraphDatabase.driver(uri=NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

    with driver.session() as session:
        # 쿼리 노드 생성
        session.run("MERGE (q:Query {text: $query_text})", query_text=user_query)

        # 문서와 엔터티 노드 및 관계 생성
        for record in graph_data:
            document_text = record['document_text']
            entities = record['entities']

            # 문서 노드 생성 및 쿼리와 연결
            session.run("""
                MERGE (d:Document {text: $document_text})
                MERGE (q:Query {text: $query_text})-[:RETURNS]->(d)
            """, document_text=document_text, query_text=user_query)

            # 엔터티 노드 생성 및 문서와 연결
            for entity in entities:
                session.run("""
                    MERGE (e:Entity {id: $entity_id})
                    MERGE (d:Document {text: $document_text})-[:MENTIONS]->(e)
                """, entity_id=entity, document_text=document_text)


def show_query_graph(query: str):
    # 쿼리 문자열에서 작은따옴표를 이스케이프 처리
    escaped_query = query.replace("'", "\\'")

    # 기본 Cypher 쿼리 정의: 쿼리와 연결된 노드와 관계를 모두 반환
    cypher_query = f"""
    MATCH (q:Query {{text: '{escaped_query}'}})-[r]->(n)
    RETURN q, r, n
    """

    # Neo4j 데이터베이스에 연결
    driver = GraphDatabase.driver(uri=NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))
    with driver.session() as session:
        graph_data = session.run(cypher_query).graph()

    # yFiles 그래프 위젯으로 시각화
    widget = GraphWidget(graph=graph_data)
    widget.node_label_mapping = 'text'
    display(widget)


In [ ]:
from langchain.prompts import PromptTemplate
from langchain.schema import HumanMessage, SystemMessage

def CHAT_EXAMPLE(query: str):
    # Cypher 쿼리에서 사용하기 위해 쿼리 문자열의 작은따옴표를 이스케이프 처리
    escaped_query = query.replace("'", "\\'")

    # Cypher 쿼리를 사용하여 그래프에서 관련 정보를 검색
    driver = GraphDatabase.driver(
        uri=NEO4J_URI,
        auth=(NEO4J_USERNAME, NEO4J_PASSWORD)
    )
    with driver.session() as session:
        # 입력된 쿼리와 관련된 엔터티를 매칭하기 위해 Cypher 쿼리 수정
        cypher_query = f"""
        MATCH (n:Document)-[:MENTIONS]->(e)
        WHERE n.text CONTAINS '{escaped_query}'
        RETURN n.text AS document_text, collect(e.id) AS entities LIMIT 3
        """
        result = session.run(cypher_query)
        graph_data = result.data()

    documents = "\n\n".join([record['document_text'] for record in graph_data])
    entities = ", ".join([entity for record in graph_data for entity in record['entities']])

    # 그래프에서 추출한 정보와 사용자 쿼리를 결합
    llm_input = f"User query: {query}\n\nRelated information from the graph:\n{documents}\n\nEntities involved: {entities}"

    # 그래프 데이터와 사용자 쿼리에 기반하여 LLM을 사용해 응답 생성
    messages = [HumanMessage(content=llm_input)]
    response = client.invoke(messages)

    llm_output = response.content
    print("Generated Response:\n", llm_output)

    # 쿼리와 관련된 정보 그래프에 저장
    store_query_in_graph(query, graph_data)

    # 저장된 그래프 시각화
    show_query_graph(query)

In [ ]:
query = "Tell me about Elizabeth I's reign."
CHAT_EXAMPLE(query)

Generated Response:
 Elizabeth I, the daughter of Henry VIII and Anne Boleyn, reigned as Queen of England from 1558 to 1603. Her reign is often referred to as the Elizabethan Era, a period marked by significant cultural, political, and economic transformation.

### Key Aspects of Elizabeth I's Reign

1. **Religious Settlement**:
   - Elizabeth established the Elizabethan Religious Settlement, which created a moderate form of Protestantism that sought to unify her kingdom and reduce religious conflict. This settlement included the Act of Supremacy (1559), which re-established the English monarch as the head of the Church of England, and the Act of Uniformity (1559), which set the order of prayer to be used in the English Book of Common Prayer.

2. **Cultural Flourishing**:
   - Her reign saw a great flowering of English culture, particularly in literature and the arts. This period, known as the English Renaissance, produced iconic figures such as William Shakespeare, Christopher Marlowe

GraphWidget(layout=Layout(height='500px', width='100%'))